In [0]:
%pip install -q -r ./../requirements.txt
dbutils.library.restartPython()

In [0]:
from operator import itemgetter
import mlflow
import os
from typing import Any, Callable, Dict, Generator, List, Optional

from databricks.vector_search.client import VectorSearchClient

from vector_search_utils.self_querying_retriever import load_self_querying_retriever
#from supervisor_utils.decomposer import load_decomposer

from databricks_langchain import ChatDatabricks
from databricks_langchain import DatabricksVectorSearch

from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    MessagesPlaceholder,
)

from langgraph_supervisor import create_supervisor
from langchain.chat_models import init_chat_model

from langchain_core.runnables import RunnablePassthrough, RunnableBranch
from langchain_core.messages import HumanMessage, AIMessage
from langchain.retrievers.multi_query import MultiQueryRetriever

from langchain_core.output_parsers import BaseOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

from mlflow.pyfunc import ChatAgent
from mlflow.types.agent import (
    ChatAgentChunk,
    ChatAgentMessage,
    ChatAgentResponse,
    ChatContext,
)

In [0]:
from dbruntime.databricks_repl_context import get_context
TOKEN = get_context().apiToken
HOSTNAME = get_context().browserHostName
USERNAME = get_context().user

In [0]:
HOSTNAME

In [0]:
os.environ['DATABRICKS_TOKEN'] = TOKEN
os.environ['DATABRICKS_URL'] = get_context().apiUrl

In [0]:
os.environ['DATABRICKS_URL']

In [0]:
from openai import OpenAI

client = OpenAI(
            api_key=os.environ["DATABRICKS_TOKEN"],
            base_url=f"{os.environ['DATABRICKS_URL']}/serving-endpoints"
        )

client.chat.completions.create(
        model="databricks-claude-sonnet-4",  # This example uses Databricks hosted Claude. If you provide your own OpenAI credentials, replace with a valid OpenAI model e.g., gpt-4o-mini, etc.
        messages=[
            {"role": "user", "content": "Hello, can you hear me?"},
        ],
        max_tokens=200,  # Max tokens for the judge's rationale
        temperature=0.0, # For more deterministic judging
    )

In [0]:
messages = [
    ChatAgentMessage(
        role="user",
        content="What was the liquidity ratio for American express in 2022? What factors contributed to this?",
    ),
    ChatAgentMessage(
        role="user",
        content="What was the liquidity ratio for American express in 2022? What factors contributed to this?",
    ),
    ChatAgentMessage(
        role="user",
        content="What was the liquidity ratio for American express in 2022? What factors contributed to this?",
    )
]

In [0]:
type(messages[0])

In [0]:
messages_final = [
  m.model_dump() for m in messages
]

messages_final

In [0]:
ca_message = ChatAgentMessage(
    role="user",
    content="What was the liquidity ratio for American express in 2022? What factors contributed to this?",
)

ca_message.dict()

In [0]:
## Enable MLflow Tracing
mlflow.langchain.autolog()

# Load the chain's configuration
model_config = mlflow.models.ModelConfig(development_config="./configs/agent.yaml")

databricks_config = model_config.get("databricks_config")
genie_config = model_config.get("genie_config")
llm_config = model_config.get("llm_config")

retriever_config = model_config.get("retriever_config")
vector_search_schema = retriever_config.get("schema")

# FM for generation
model = ChatDatabricks(
    endpoint=llm_config.get("llm_endpoint_name"),
    extra_params=llm_config.get("llm_parameters"),
)

In [0]:
from vector_search_utils.self_querying_retriever import load_self_querying_retriever

sq_retriever = load_self_querying_retriever(model, databricks_config, retriever_config)

research_agent = create_react_agent(
    model=model,
    tools=[sq_retriever.as_tool()],
    prompt=(
        "You are a research agent.\n\n"
        "INSTRUCTIONS:\n"
        "- After you're done with your tasks, respond to the supervisor directly\n"
        "- Respond ONLY with the results of your work, do NOT include ANY other text."
    ),
    name="research_agent",
)

In [0]:
type(research_agent)

In [0]:
from databricks.sdk import WorkspaceClient

from databricks_langchain.genie import GenieAgent
from langgraph.prebuilt import create_react_agent

class GenieInput(BaseModel):
    """OpenAI-format chat history for Genie."""
    messages: List[Dict[str, Any]] = Field(
        ..., description="Chat messages in {'role','content'} format"
    )

genie_agent_tool = GenieAgent(
    genie_space_id="01f047a4c99217ed9a6b2dcee6a805cb",
    genie_agent_name="SEC_genie_agent",
    description=genie_config.get("genie_space_description"),
    client=WorkspaceClient(),
).as_tool(
        name="sec_genie",
        description="Ask Genie questions about public-company filings and metrics",
        args_schema=GenieInput,   # <- avoids the NoneType error you hit earlier
)

In [0]:
react_agent = create_react_agent(
    model=model,
    tools=[genie_agent_tool],
    prompt="You are a meticulous financial analyst. "
           "Think step-by-step; call tools when needed."
)

In [0]:
example_input = {
        "messages": [
            {
                "role": "user",
                "content": "What was the liquidity ratio for American express in 2022? What factors contributed to this?",
            }
        ]
    }
    
react_agent.invoke(example_input)

In [0]:
example_input = {
        "messages": [
            {
                "role": "user",
                "content": "What was the liquidity ratio for American express in 2022? What factors contributed to this?",
            }
        ]
    }

In [0]:
example_input = {
        "input": [
            {
                "role": "user",
                "content": "What was the liquidity ratio for American express in 2022? What factors contributed to this?",
            }
        ]
    }

In [0]:
genie_agent_tool.get_graph()

In [0]:
genie_agent_tool.invoke(example_input)

In [0]:
genie_agent = create_react_agent(
    model=model,
    tools=[genie_agent_tool],
    prompt=(
        "You are a financial metrics analyst agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Your task is query a sql database containing financial data for different sec companies"
        "- After you're done with your tasks, respond to the supervisor directly\n"
        "- Respond ONLY with the results of your work, do NOT include ANY other text."
    ),
    name="metrics_analyst",
)

In [0]:
genie_agent.invoke(example_input)